# Fed AVeraging 

Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, utils, datasets
from torchsummary import summary

Data Analysis

In [2]:
data = pd.read_csv("/content/feature_selected_voice_data.csv")
print(data.shape)         # Lookiing the shape of the data
print(data.columns)       # Looking how many columns data has
data.dtypes  
data.head()

(3168, 14)
Index(['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'kurt', 'sp.ent',
       'sfm', 'mode', 'meanfun', 'minfun', 'modindx', 'label'],
      dtype='object')


,meanfreq,sd,median,Q25,Q75,IQR,kurt,sp.ent,sfm,mode,meanfun,minfun,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,274.402906,0.893369,0.491918,0.000000,0.084279,0.015702,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,634.613855,0.892193,0.513724,0.000000,0.107937,0.015826,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,1024.927705,0.846389,0.478905,0.000000,0.098706,0.015656,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,4.177296,0.963322,0.727232,0.083878,0.088965,0.017798,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,4.333713,0.971955,0.783568,0.104261,0.106398,0.016931,0.208274,male


In [3]:
data["label"].value_counts()

male      1584
female    1584
Name: label, dtype: int64

In [4]:
X = data.iloc[:,:-1]
Y = data["label"]

In [5]:
# Encoding the labels in digits}
le = preprocessing.LabelEncoder()
Y = le.fit_transform(Y)

In [6]:
sc = StandardScaler()  
X = sc.fit_transform(X)      
X = sc.transform(X)

In [7]:
n_samples, n_features = X.shape

In [8]:
# Splitting the Dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42, stratify = Y)
print((y_train == 1).sum())
print((y_train == 0).sum())

1109
1108


In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [10]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train =torch.from_numpy(y_train)
y_test =torch.from_numpy(y_test)

Logistic Regression Model

In [11]:
# logistic regression class
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    #sigmoid transformation of the input 
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [12]:
lr = LogisticRegression(n_features)

In [14]:
num_epochs = 500
learning_rate = 0.0001 
criterion = nn.BCELoss() # Binary cross Entropy loss                              
optimizer = torch.optim.SGD(lr.parameters(), lr=learning_rate)  

In [18]:
import torch
class Foo:
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return self.data.shape

# myFoo = Foo(data=torch.rand(10, 20))
# print(len(myFoo))

In [21]:
y_pred = lr(X_train)
print(y_pred.size())
print(y_train.size())

torch.Size([2217, 1])
torch.Size([2217])


In [24]:
yy = y_pred.reshape([2217])

print(yy.size())

torch.Size([2217])


In [25]:
error_loss = []
for epoch in range(num_epochs):
    train_loss = 0
    optimizer.zero_grad()
    y_pred = lr(X_train)
    loss = criterion(y_pred.reshape(2217), y_train)             
    loss.backward()
    optimizer.step()
    if (epoch+1) % 20 == 0:                                          
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
    train_loss += loss.item()*X_train.size(0)
    error_loss.append(train_loss)
total_loss = sum(error_loss)/len(error_loss)

print(lr.parameters()) # printing learnable parameters (weights and biases)
print(lr.state_dict) # pint dictionary of all parameters
print(total_loss) # printing total loss in training the model

epoch: 20, loss = 6.7009
epoch: 40, loss = 4.6716
epoch: 60, loss = 3.4812
epoch: 80, loss = 2.6198
epoch: 100, loss = 2.1980
epoch: 120, loss = 1.8175
epoch: 140, loss = 1.4919
epoch: 160, loss = 1.2821
epoch: 180, loss = 1.0638
epoch: 200, loss = 0.9094
epoch: 220, loss = 0.7781
epoch: 240, loss = 0.6700
epoch: 260, loss = 0.5816
epoch: 280, loss = 0.5095
epoch: 300, loss = 0.4507
epoch: 320, loss = 0.4024
epoch: 340, loss = 0.3627
epoch: 360, loss = 0.3299
epoch: 380, loss = 0.3027
epoch: 400, loss = 0.2798
epoch: 420, loss = 0.2606
epoch: 440, loss = 0.2443
epoch: 460, loss = 0.2306
epoch: 480, loss = 0.2190
epoch: 500, loss = 0.2093
<generator object Module.parameters at 0x7f9625c266d0>
<bound method Module.state_dict of LogisticRegression(
  (linear): Linear(in_features=13, out_features=1, bias=True)
)>
3267.3448665813803


In [27]:
print(lr.parameters()) 

<generator object Module.parameters at 0x7f9625be92d0>


In [34]:
for name, param in lr.named_parameters():
    if name in ['fc.weight', 'fc.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [36]:
for name in lr.parameters():
    print(name, ':', param.requires_grad)

Parameter containing:
tensor([[-0.0068, -0.0098,  0.1006,  0.0747, -0.0039,  0.1627,  0.2252,  0.1600,
         -0.0861, -0.1513, -0.2190, -0.0024, -0.1800]]) : False
Parameter containing:
tensor([0.0897]) : False


In [26]:
print(lr.state_dict)

<bound method Module.state_dict of LogisticRegression(
  (linear): Linear(in_features=13, out_features=1, bias=True)
)>


Local Training on Client Update

In [ ]:
# Loading Dataset
# class CustomDataset(Dataset):
#   def __init__(self, dataset, idxs):
#       self.dataset = dataset
#       self.idxs = list(idxs)

#   def __len__(self):
#       return len(self.idxs)

#   def __getitem__(self, item):
#       image, label = self.dataset[self.idxs[item]]
#       return image, label

In [ ]:
# Client Side Training
class ClientUpdate(object):
  def __init__(self, dataset, learning_rate, epochs, idxs):
    self.train_loader = DataLoader(dataset, shuffle=True)
    self.learning_rate = learning_rate
    self.epochs = epochs

    criterion = nn.BCELoss()                                
    optimizer = torch.optim.SGD(lr.parameters(), lr=learning_rate)  

    error_loss = []
    for epoch in range(self.epochs):
      train_loss = 0
      y_pred = lr(X_train)
      for data, labels in self.train_loader:
        if torch.cuda.is_available():
          data, labels = data.cuda(), labels.cuda()
        optimizer.zero_grad()
        loss = criterion(y_pred, y_train)             
        loss.backward()
        optimizer.step()

        if (epoch+1) % 20 == 0:                                         
            print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        train_loss += loss.item()*data.size(0)
        error_loss.append(train_loss)

    total_loss = sum(error_loss)/len(error_loss)
    return lr.state_dict(), total_loss

In [ ]:
# Server Side Training
import time
import copy
import matplotlib.pyplot as plt

def Server_training(model, rounds, lr, ds, C, K, E):

  global_weights = model.state_dict()
  train_loss = []
  start = time.time()

  for curr_round in range(1, rounds+1):
    w, local_loss = [], []

    m = max(int(C*K), 1)
    
    S_t = np.random.choice(range(K), m, replace=False)
    for k in S_t:
      local_update = ClientUpdate(dataset=ds, learning_rate=lr, epochs=E)
      weights, loss = local_update.train(model=copy.deepcopy(model))

      w.append(copy.deepcopy(weights))
      local_loss.append(copy.deepcopy(loss))

    # updating the global weights
    weights_avg = copy.deepcopy(w[0])
    for k in weights_avg.keys():
      for i in range(1, len(w)):
        weights_avg[k] += w[i][k]

      weights_avg[k] = torch.div(weights_avg[k], len(w))

    global_weights = weights_avg

    # move the updated weights to our model state dict
    model.load_state_dict(global_weights)

    # loss
    loss_avg = sum(local_loss) / len(local_loss)
    print('Round: {}... \tAverage Loss: {}'.format(curr_round, round(loss_avg, 3)))
    train_loss.append(loss_avg)

  end = time.time()
  fig, ax = plt.subplots()
  x_axis = np.arange(1, rounds+1)
  y_axis = np.array(train_loss)
  ax.plot(x_axis, y_axis, 'tab:'+'r')

  ax.set(xlabel='Number of Rounds', ylabel='Train Loss',
       title="TrainLoss vs Epochs plot")
  ax.grid()
  fig.savefig('TrainLoss_vs_Epochs_plot".jpg', format='jpg')
  print("Training Done!")
  print("Total time taken to Train: {}".format(end-start))
  return model

